In [1]:
import pandas as pd
import numpy as np
from chinese_calendar import is_workday

In [2]:
data_first = pd.read_csv('./order_train1.csv')
data_first=data_first.drop_duplicates(keep='first')

In [3]:
def set_index(data_first):
    data_second = data_first.iloc[:,:]
    data_second=data_second.rename(columns={'order_date':'订单日期','sales_region_code':'销售区域编码','item_code':'产品编码',
                                           'first_cate_code':'产品大类编码','second_cate_code':'产品细类编码',
                                            'sales_chan_name':'销售渠道名称','item_price':'产品价格','ord_qty':'订单需求量'})
#     data_second.columns=['订单日期','销售区域编码','产品编码','产品大类编码','产品细类编码','销售渠道名称','产品价格','订单需求量']
    return data_second

In [4]:
data_second = set_index(data_first)
data_second.isna().sum()

订单日期      0
销售区域编码    0
产品编码      0
产品大类编码    0
产品细类编码    0
销售渠道名称    0
产品价格      0
订单需求量     0
dtype: int64

In [5]:
data_second.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 597382 entries, 0 to 597693
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   订单日期    597382 non-null  object 
 1   销售区域编码  597382 non-null  int64  
 2   产品编码    597382 non-null  int64  
 3   产品大类编码  597382 non-null  int64  
 4   产品细类编码  597382 non-null  int64  
 5   销售渠道名称  597382 non-null  object 
 6   产品价格    597382 non-null  float64
 7   订单需求量   597382 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 41.0+ MB


In [6]:
# 读取数据集
data = data_second.copy(deep=True)

# 将订单日期转换为datetime格式
data['订单日期'] = pd.to_datetime(data['订单日期'], format='%Y/%m/%d')

# 新增属性列“季节”
data['季节'] = data['订单日期'].apply(lambda x: 
                                    1 if x.month in [3, 4, 5] 
                                    else (2 if x.month in [6, 7, 8] 
                                    else (3 if x.month in [9, 10, 11] 
                                    else 4)))

# 新增属性列“月时间段”
data['月时间段'] = data['订单日期'].apply(lambda x: 
                                1 if x.day <= 10 
                                else (2 if x.day <= 20 
                                else 3))

# 是否为线上线下
data['销售渠道名称'] = data['销售渠道名称'].apply(lambda x: 
                                1 if x == 'online' 
                                else 0 
                                )

#属性列 '促销期'
data['促销期'] = 0
for i in range(np.sort(list(set(data['订单日期'].dt.year)))[0],np.sort(list(set(data['订单日期'].dt.year)))[-1]+1):
        data.loc[(data['订单日期'] >= '%d-03-01'%i) & (data['订单日期'] <= '%d-03-08'%i), '促销期'] = 38
        data.loc[(data['订单日期'] >= '%d-06-01'%i) & (data['订单日期'] <= '%d-06-20'%i), '促销期'] = 618
        data.loc[(data['订单日期'] >= '%d-10-01'%i) & (data['订单日期'] <= '%d-10-07'%i), '促销期'] = 101
        data.loc[(data['订单日期'] >= '%d-11-01'%i) & (data['订单日期'] <= '%d-11-11'%i), '促销期'] = 1111
        data.loc[(data['订单日期'] >= '%d-12-01'%i) & (data['订单日期'] <= '%d-12-12'%i), '促销期'] = 1212
        data.loc[(data['订单日期'] >= '%d-12-24'%i) & (data['订单日期'] <= '%d-12-25'%i), '促销期'] = 1205
        data.loc[(data['订单日期'] >= '%d-01-01'%i) & (data['订单日期'] <= '%d-02-10'%i), '促销期'] = 1

#属性列 '是否为节假日'
holidays = []
for year in range(np.sort(list(set(data['订单日期'].dt.year)))[0], np.sort(list(set(data['订单日期'].dt.year)))[-1]+1):
    holidays.append(pd.DataFrame({'订单日期': pd.date_range(start=str(year)+'-01-01', end=str(year)+'-12-31')}))
holidays = pd.concat(holidays)
holidays['is_holiday'] = holidays['订单日期'].apply(lambda x: not is_workday(x))
holidays['is_holiday'] = holidays['is_holiday'].apply(lambda x: 
                                                     1 if x==True
                                                     else (0))
# 按照订单日期将数据集合并
data = pd.merge(data, holidays, on='订单日期', how='left')
data = data.rename(columns = {'is_holiday':'是否为节假日'})
data['year']=data['订单日期'].dt.year
data['month']=data['订单日期'].dt.month
data['day']=data['订单日期'].dt.day
cols = data.columns.tolist()
cols.append(cols.pop(cols.index('订单需求量')))
data = data.reindex(columns=cols)
print(data)
data.to_csv('./数据预处理后的文件.csv')
# 输出结果


             订单日期  销售区域编码   产品编码  产品大类编码  产品细类编码  销售渠道名称    产品价格  季节  月时间段  \
0      2015-09-01     104  22069     307     403       0  1114.0   3     1   
1      2015-09-01     104  20028     301     405       0  1012.0   3     1   
2      2015-09-02     104  21183     307     403       1   428.0   3     1   
3      2015-09-02     104  20448     308     404       1   962.0   3     1   
4      2015-09-02     104  21565     307     403       0  1400.0   3     1   
...           ...     ...    ...     ...     ...     ...     ...  ..   ...   
597377 2018-12-20     102  20994     302     408       0   843.0   4     2   
597378 2018-12-20     102  21875     302     408       0   762.0   4     2   
597379 2018-12-20     102  20215     302     408       0  2013.0   4     2   
597380 2018-12-20     102  20195     302     408       0  2120.0   4     2   
597381 2018-12-20     102  20321     302     408       0  1244.0   4     2   

        促销期  是否为节假日  year  month  day  订单需求量  
0         0     

In [7]:
data.columns

Index(['订单日期', '销售区域编码', '产品编码', '产品大类编码', '产品细类编码', '销售渠道名称', '产品价格', '季节',
       '月时间段', '促销期', '是否为节假日', 'year', 'month', 'day', '订单需求量'],
      dtype='object')